## Setup & imports

In [ ]:
# Setup: make "backend/" importable as a package root
import sys, pathlib
ROOT = pathlib.Path(__file__).resolve().parents[1]  # backend/
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

# Core imports
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from app.services.backtest import (
    fetch_prices, add_features, make_windows, build_model
)
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import matplotlib.pyplot as plt

## Parameters

In [ ]:
TICKER   = "AAPL"     # any equity/ETF/crypto (crypto often like "BTC-USD")
CONTEXT  = 100        # last N sessions used as the context window
LOOKBACK = 60         # <= 80 so we can train with 80 sessions for backtest
H_BACK   = 20         # backtest horizon (predict last 20 sessions)
H_FORE   = 10         # forecast horizon (predict next 10 sessions)

# (Optional) output
SAVE_JSON = True
OUT_JSON  = "unified_backtest_forecast.json"

## Fetch data & build features (reusing service helpers)

In [ ]:
# Overfetch calendar days so we reliably get >= CONTEXT + LOOKBACK + 40 sessions after dropna
overfetch_days = CONTEXT + LOOKBACK + 200
start_date = (datetime.utcnow() - timedelta(days=overfetch_days)).strftime("%Y-%m-%d")

raw = fetch_prices(TICKER, start=start_date, end=None, interval="1d")
feat = add_features(raw)

# Keep only the last CONTEXT sessions for the combined view
if len(feat) < CONTEXT + 5:
    raise ValueError(f"Not enough data. Need at least ~{CONTEXT+5} feature rows, got {len(feat)}")

ctx = feat.iloc[-CONTEXT:].copy()
ctx.index = pd.to_datetime(ctx.index)

# Targets = next-session log-return
ctx["target_ret"] = ctx["log_ret"].shift(-1)
ctx = ctx.dropna()

## One-shot backtest on the last 20 sessions

In [ ]:
# Training slice for backtest = everything up to the last H_BACK rows
start_idx = len(ctx) - H_BACK
if start_idx <= LOOKBACK:
    raise ValueError(f"Training slice too short for LOOKBACK={LOOKBACK}. "
                     f"Have {start_idx} training rows. Lower LOOKBACK or increase CONTEXT.")

X_df = ctx[[
    "Close","Volume","log_ret","ret",
    "roll_mean_7","roll_std_7","roll_mean_21","roll_std_21",
    "rsi_14","macd","macd_signal","macd_diff",
    "bb_width","ret_lag1","ret_lag3","ret_lag5",
    "vol_7","vol_21","z_close_21",
]].astype("float32")
y = ctx["target_ret"].astype("float32").values
dates = ctx.index

# Fit scalers on train slice only
train_X_df = X_df.iloc[:start_idx]
train_y    = y[:start_idx]

scaler_X = StandardScaler().fit(train_X_df.values)
scaler_y = StandardScaler().fit(train_y.reshape(-1,1))
X_train  = scaler_X.transform(train_X_df.values)
y_train_s = scaler_y.transform(train_y.reshape(-1,1)).ravel()

# Build windows and train
Xw, yw = make_windows(X_train, y_train_s, LOOKBACK, H_BACK)
if len(Xw) == 0:
    raise ValueError("No training windows for backtest. Adjust LOOKBACK/H_BACK.")

model_back = build_model(LOOKBACK, Xw.shape[2], H_BACK)
model_back.fit(Xw, yw, epochs=24, batch_size=32, verbose=0)

# Predict last H_BACK from final LOOKBACK training rows
last_block_raw = train_X_df.values[-LOOKBACK:]
last_block = scaler_X.transform(last_block_raw).reshape(1, LOOKBACK, Xw.shape[2])
next_ret_s = model_back.predict(last_block, verbose=0)[0]
next_ret   = scaler_y.inverse_transform(next_ret_s.reshape(-1,1)).ravel()

# Convert returns to prices
last_price_bt = float(ctx["Close"].iloc[start_idx-1])
pred_back_prices = last_price_bt * np.exp(np.cumsum(next_ret))
actual_back = ctx["Close"].iloc[start_idx : start_idx + H_BACK].values.astype("float64")

back_dates = dates[start_idx : start_idx + H_BACK]
back_df = pd.DataFrame({
    "date": back_dates,
    "actual": actual_back,
    "pred_back": pred_back_prices[:len(actual_back)]
}).set_index("date")

## One-shot forecast next 10 sessions

In [ ]:
# Retrain on full CONTEXT slice and predict H_FORE forward
scaler_X2 = StandardScaler().fit(X_df.values)
scaler_y2 = StandardScaler().fit(y.reshape(-1,1))
X_all  = scaler_X2.transform(X_df.values)
y_all_s = scaler_y2.transform(y.reshape(-1,1)).ravel()

Xw2, yw2 = make_windows(X_all, y_all_s, LOOKBACK, H_FORE)
if len(Xw2) == 0:
    raise ValueError("No training windows for forecast. Adjust LOOKBACK/H_FORE/CONTEXT.")

model_fore = build_model(LOOKBACK, Xw2.shape[2], H_FORE)
model_fore.fit(Xw2, yw2, epochs=24, batch_size=32, verbose=0)

last_block_raw2 = X_df.values[-LOOKBACK:]
last_block2 = scaler_X2.transform(last_block_raw2).reshape(1, LOOKBACK, Xw2.shape[2])
next_ret_s2 = model_fore.predict(last_block2, verbose=0)[0]
next_ret2   = scaler_y2.inverse_transform(next_ret_s2.reshape(-1,1)).ravel()

last_price = float(ctx["Close"].iloc[-1])
pred_fore_prices = last_price * np.exp(np.cumsum(next_ret2))

# Build future dates: business days for equities, daily for crypto-like tickers
last_dt = ctx.index[-1]
is_crypto = TICKER.upper().endswith("-USD")
freq = "D" if is_crypto else "B"
future_idx = pd.date_range(last_dt + pd.tseries.frequencies.to_offset("D"), periods=H_FORE, freq=freq)

fore_df = pd.DataFrame({
    "date": future_idx,
    "pred_fore": pred_fore_prices
}).set_index("date")

## Merge + metrics + plot (single chart)

In [ ]:
# Merge actuals (last CONTEXT) with predicted segments
actual_df = ctx[["Close"]].rename(columns={"Close":"actual"}).iloc[-CONTEXT:]
combined = actual_df.join(back_df[["pred_back"]], how="left")

# Append forecast rows (no actuals there)
combined = pd.concat([combined, fore_df], axis=0)

# Simple friendly metrics on backtest window
# Accuracy = directional hit-rate comparing day-to-day changes
if len(back_df) > 1:
    da = back_df["actual"].diff().to_numpy()
    dp = back_df["pred_back"].diff().to_numpy()
    valid = np.isfinite(da) & np.isfinite(dp)
    hits = (np.sign(da[valid]) == np.sign(dp[valid])).sum()
    denom = valid.sum()
    accuracy_pct = (hits / denom * 100) if denom else 0.0
else:
    accuracy_pct = 0.0

mae_dollar = float(mean_absolute_error(back_df["actual"], back_df["pred_back"])) if len(back_df) else 0.0
mape_pct   = float(mean_absolute_percentage_error(back_df["actual"], back_df["pred_back"]) * 100) if len(back_df) else 0.0
rmse       = float(np.sqrt(mean_squared_error(back_df["actual"], back_df["pred_back"])) ) if len(back_df) else 0.0

metrics = {
    "ticker": TICKER,
    "context_sessions": CONTEXT,
    "lookback": LOOKBACK,
    "backtest_horizon": H_BACK,
    "forecast_horizon": H_FORE,
    "accuracy_pct": accuracy_pct,
    "avg_abs_error": mae_dollar,
    "mape_pct": mape_pct,
    "rmse": rmse,
}

metrics

if SAVE_JSON:
    import json
    payload = {
        "metrics": metrics,
        "series": combined.reset_index().assign(date=combined.index.strftime("%Y-%m-%d")).to_dict(orient="records"),
    }
    with open(OUT_JSON, "w") as f:
        json.dump(payload, f, indent=2)

# Plot: one chart with (1) actuals (last 100), (2) backtest preds (last 20), (3) forecast preds (next 10)
plt.figure(figsize=(11,5))
combined["actual"].plot(label="Actual (last 100)", lw=2, color="black")

# Backtest predicted segment (overlapping last 20)
combined["pred_back"].plot(label="Predicted (backtest last 20)", lw=2, color="tab:orange")

# Forecast segment (future 10)
combined["pred_fore"].plot(label="Predicted (forecast next 10)", lw=2, linestyle="--", color="tab:blue")

plt.title(f"{TICKER} — Unified Backtest (20) + Forecast (10) using last {CONTEXT} sessions, lookback={LOOKBACK}")
plt.xlabel("Date"); plt.ylabel("Price")
plt.legend()
plt.tight_layout()
plt.show()